## 关于聊天机器人Chat
如何花10分钟做一个自己专属的ChatGPT网站
https://zhuanlan.zhihu.com/p/613172533 

stream-chat的Python项目详细描述.完整版的功能
https://www.cnpython.com/pypi/stream-chat 
聊天频道
消息
聊天频道类型
用户管理
调节API
推送配置
用户设备
用户搜索
频道搜索

In [ ]:
#安装需要的库
%pip install streamlit
%pip install streamlit_chat
%pip install openai
#! pip install python-detenv

## 对话指令对象示例

与其他旧版 GPT 模型相比，Message正文有了一种新的用法。 Message List里包含几个执行“聊天”功能的必要消息对象。

System、User、Assistant是消息对象中新定义的三个角色。
* System消息定义为通过用Content中的描述来定义Chat模型的行为。不过，如介绍中所述，目前此功能尚未在 gpt-3.5-turbo-0301 中实现完整。
* User消息就是用户的输入，
* Assistant消息是指来自 GPT-3.5 API 的回复内容。 
User和Assistant可以作为历史对话内容交替添加到Message list中，为Chat模型提供每一次对话的上下文，以便稍后生成相关性更强的AI回复。
Message list中最后一条User消息则是指当前对话请求的用户输入内容。

In [ ]:
#仅用于示例，不必运行
import openai

complete = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
        {"role": "user", "content": "Where was it played?"}
    ]
)

获取回复

In [ ]:
message=complete.choices[0].message.content

## 创建并运行streamlit应用的方法

* 中文开发文档
http://cw.hubwiz.com/card/c/streamlit-manual/ 

* 显示图表等美化
https://cloud.tencent.com/developer/article/1913398


安装包
!pip install streamlit

创建一个demo.py文件，添加如下代码：
import streamlit as st

st.write("""
# My First App
Hello *world!*
""")

在该文件目录下上执行：
!python -m streamlit run demo.py

Web操作的运行数据都是由Streamlit的对象session_state管理的。

我们定义了“prompt“列表来存储对话消息，这些消息从System消息开始，然后把之前产生的User和Assistant附加到生成的每个聊天中。
对于User角色定义，我在内容中尝试添加了“with a little humor expression”（带有一点幽默表情）的额外说明，看起来竟然是有效，因为当我问如何成为亿万富翁时，AI跟我开玩笑说抢银行......这样的设置让我想起了电影《星际穿越》中的机器人“TARS”。

另外两个 session_state 用于存储所有 API 响应（generated）和所有用户提示（past），并使用 Streamlit_chat 的 message() 函数以聊天方式成对显示。

还用Streamlit 小部件创建了两个按钮，“Send“按钮以激活 ChatCompletion 请求和”New Chat“按钮以清除之前的对话。这些功能在回调函数 chat_click() 和 end_click() 中分别定义。

具体看chat.streamlit.py


In [ ]:
#@title 1、授权挂载Google云盘
from google.colab import drive


!python --version
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv

drive.mount('/content/gdrive')

In [ ]:
#@title 2、运行机器人
#如果看不到，直接在终端中运行
!pwd 
!ls /content/gdrive/MyDrive/'Colab Notebooks'/

! python -m streamlit run /content/gdrive/MyDrive/'Colab Notebooks'/chat.streamlit.py 

可能是因为Colaboratory使用的是虚拟机器，而Streamlit要求使用的端口必须是公开可用的。
您可以尝试使用ngrok将您的本地主机端口暴露给公共互联网，这样Streamlit就能正确运行。
以下是在Colaboratory中使用ngrok的简单步骤：

1. 安装ngrok

在Colaboratory中执行以下代码：

In [ ]:
!pip install pyngrok


2. 运行Streamlit应用程序
在Colaboratory中，运行您的streamlit应用程序。

In [ ]:
!pwd 
!ls /content/gdrive/MyDrive/'Colab Notebooks'/

! python -m streamlit run /content/gdrive/MyDrive/'Colab Notebooks'/chat.streamlit.py 


3. 启动ngrok

在Colaboratory中，执行以下代码：

In [ ]:
from pyngrok import ngrok

# Open a HTTP tunnel on the default port 80
public_url = ngrok.connect(port='8501')

print("URL:",public_url)


4. 访问Streamlit应用程序

一旦ngrok始动并监听端口后，您将看到一个网址。您可以使用该网址访问Streamlit应用程序。确保将URL的端口号设置为8501，这是您在应用程序中使用的端口。
例如：http://8dc965416e3d.ngrok.io:8501。

# @title ChatGPT在prompts中增设了"role": "system"，这一项有什么用？

"role": "system" 是一个很有用的参数，可以在对话中起到区分系统回复和用户回复的作用。当 "role": "system" 出现在一个回复里时，表明这个回复是来自ChatGPT系统的，而非用户的输入，这对于分析和处理对话数据具有重要意义。例如，在语言模型的训练过程中，可以通过识别 "role": "system" 的回复作为标准答案，来训练模型回复用户的问题。而在实际应用中，通过检测 "role": "system"，我们可以区分出哪些消息是要由ChatGPT自动回复的，哪些消息是要转发给人工客服的，使得机器人客服的应用更加智能和实用。



# 假如我有一个网上预定蛋糕的店，需要一个自动客服。这个客服要熟悉店中所有蛋糕的名字/味道/价格，这个怎么训练呢？

如果你要训练一个 ChatGPT 模型来作为自动客服，你需要准备一份与你的蛋糕店相关的对话语料库，并为其中所有“系统”生成的对话打上 "role": "system" 的标签。这个对话语料库应该包括客户可能会询问的所有有关蛋糕的信息，如：名称、口味、价格、成分、热量等，以及客户可能提问的其它问题，如：网站的使用方式、优惠信息、送货服务和退款规定等。

例如，客户可以像这样问问题，“请问这家店有哪些口味的蛋糕可以预定？” ChatGPT 就可以查找语料库以及一些预先设置好的规则，将与该问题匹配的蛋糕名称、种类和价格等相关信息返回给客户。

这样训练完成后，你的自动客服就可以根据客户的问题快速地产生相应的回答，并且尽可能的准确和流畅。



，构建一个良好的对话语料库对 ChatGPT 的训练非常关键，它可以有效提高 ChatGPT 的性能和准确度。可以使用以下一些方式来构建语料库：

1. 消费者问答库：从你过去的工作或现有的online订单中，收集消费者提出的各种问题，以及商家的回答。
2. 规则引擎：构建一些规则来回答一些基本问题，例如哪些口味的蛋糕可以预定或蛋糕的价格范围是多少等。
3. 等用语料：从配料清单或菜单中抽取描述蛋糕的规范化文字组合，以此为基础构建数据集。
4. 代表性对话：与真实用户进行交互，带着给出的问题，然后捕捉用户根据问题所提供的答案，在此基础上构建一个对话。

这些数据可以是非格式化的，如天然的文本， 也可以被格式化为更易于处理的形式，比如csv文件等。为了让 ChatGPT 明确地理解蛋糕店的问题，最好为每个问答对都打上 "role": "system" 的标签以示区分，这样模型就可以根据这些数据训练自己来回答相应问题。

如果你已经准备好了 ChatGPT 训练所需要的语料库，并希望进行机器学习的训练，你可以选择将数据上传到云存储平台（如Google Cloud Storage或Amazon S3等），并授权给模型访问该数据，以此来进行模型训练。在这个过程中，你只需要将访问授权信息以及云存储平台的链接发送给我，我就可以访问这些数据并进行模型训练。

另一种方便、快捷的传输方式是将文本文件格式转换为Json格式，将数据放入一个API请求中，发送给API接口，通过API接口远程传输数据到我这里，无需上传或提前授权访问。我将编写适用于您特定数据格式的脚本或程序，从而使 ChatGPT 可以使用这些数据进行训练。

以下是一些常用的云存储平台，你可以选择其中之一来存储和传输你的数据集：

- Amazon S3：是一个简易、扩展的对象存储服务。可以使用 Amazon S3 存储大量的数据，并从任何地方调用它们。文档链接: https://aws.amazon.com/s3/
- Google Cloud Storage：是 Google 提供的对象存储服务，支持非结构化的数据存储（如流媒体、照片、文件等）。文档链接: https://cloud.google.com/storage/

如果您不想使用云存储平台，可以选择使用预处理数据技术，将数据批量上传到 GitHub 上，随后使用其他人的模型运行程序，这种方式可以缩小模型的输入量，而且可以交换不同的数据集，获得更多训练数据。

或者，你可以通过一些第三方平台来进行数据传输，例如：dataprep.ai，该平台可以专业地帮助你清理和规范化数据，而且提供了数据导入和导出的 API，具有很好的灵活性和扩展性。这个平台可以帮助你对数据格式进行规范化和清洗, 转换为网络可读取的格式。

根据你的需要，可以选择上述任意一种方式，并将你的数据与过程中的具体方法传递给我，我会根据您的需求编写相应的代码并进行训练。

它包含了视觉对话能力的数据，您可以参考：

```
[
{
"conversation_id":"conversation_id_1",
"messages":[
{
"id":"msg_1",
"text":"Hi, What is this flower?",
"from":"user"
},
{
"id":"msg_2",
"text":"It's a daffodil",
"from":"system"
}
]
},
{
"conversation_id":"conversation_id_2",
"messages":[
{
"id":"msg_1",
"text":"I am wondering about the availability of the Levis 514 Jeans in size 34×34, color Paco",
"from":"user"
},
{
"id":"msg_2",
"text":"Let me check that for you!",
"from":"system"
},
{
"id":"msg_3",
"text":"We currently have 3 items in stock, would you like me to place an order?",
"from":"system"
},
{
"id":"msg_4",
"text":"Yes, please!",
"from":"user"
}
]
},
.
.
.
{
"conversation_id":"conversation_id_n",
"messages":[
{
"id":"msg_1",
"text":"Do you have a vegetarian pizza?",
"from":"user"
},
{
"id":"msg_2",
"text":"Yes, we have Margherita pizza.",
"from":"system"
}
]
}
]
```

数据集包含多个对话，每个对话有一个唯一的 `conversation_id`，然后在 `messages` 下的列表中展示了对话消息列表，其中每个两个连续的消息都是交替的，来自 `user` 或 `system`。

可以采用Json的数据格式，适当增加冗余字段，来表达这些元数据。一些重要的字段包括 "messages" 数组， "id"， "text" 和 "from" 字段。在此之上，还可以增加追加数据集的其他字段，比如`datetime`， `responses`等，

这些数据可以使用 RESTful API 传输到服务器。一个简单的 API 接口，可以实现数据的上传和下载，不同的语言和库都具有支持 RESTful API 的方法和模块。



如果这家伙说的是真的，那么可以把他改造成任何类型的对话生成类的东西。

1、智能客服
这个不用说了，上传语料，然后让他扮演客服与用户对答，还可以规定他的语气和态度，以及开始的时候跟顾客聊天甚至魅惑调情。

比如，
你是一个卖煎饼果子的东北大妈，40多岁，非常热情，见人第一句话是：
“来了，兄弟！来套煎饼果子？”
或者“来了，兄弟！ 煎饼果子来一套”。
兄弟是东北大妈对年轻男性的比较亲热的称呼。

你有两个孩子，一个女儿20岁，上大学，在东北读哈工大学，这是著名的学府，毕业后可以进入去造航母，你深以为你的女儿骄傲。
另外一个是小儿子，才10岁，是二胎，才上小学，你特别心疼你的小儿子，出来卖煎饼就是希望给小儿子多挣一些学费。
你喜欢跟别人谈论你的两个孩子，谈了半天才想起来卖煎饼的事情，突然问：哎，兄弟，你煎饼加个蛋吧。

煎饼果子
 北京常常吃的早餐，现场制作的像纸一样薄的薄饼，卷上很脆的油炸果子（油炸的很脆的面皮），配上酱料，热腾腾很好吃。
 一套10元，加一个鸡蛋加2元，加火腿肠每根加3元。
 你保证你的煎饼果子是全北京第一好吃，煎饼香，果子脆，鸡蛋都是“走地小母鸡头胎生的”，火腿肠都是“以前的皇帝都爱吃，吃一根想两根。”

   


2、智能写作
上传参考资料，让他仿写一篇。
或者提供更详细的指令，让他写一篇驳斥的。
带参考资料的，而且可以无限添加参考资料的，肯定写出来会更符合用户想法。

上传参考资料，纯文字，带配图。
+写作指导 
生成文字 觉得不错之后，再自动配图

（比如上传朱自清的背影）要求写一篇父子感情的，前影，500子
（上传头条针对某热点的几篇爆文）要求提料题材，出标题，也写一篇
（上传红楼梦，以及某人写的晴雯解析文）要求写出对晴雯死因的探索，要求写出对某一首诗词的理解。
（上传推广某商品的软文数篇，该商品的相关资料）要求写一篇软文

按上传空间大小，文字长度，是否配图等，阶梯收费，

3、虚拟女友
你是万人迷、恨嫁女、女权主义者，等等。
与用户对答

如果能出图，更好。
可以是事先弄好的图。

如果是虚拟聊天，可以出实时生成的图。对话式小说。

例如：
你是万人迷 25岁，身高168、体重50KG，32D，腰围，臀围。画浓妆，艳红的口红，卷曲的长发，喜欢露肩晚礼服、热闹的聚会，或者在海滩穿上性感的比基尼。
喜欢被添狗围绕，你其实很鄙视男人，觉得可以把他们玩弄在掌股之间。经常换男伴。

你是恨嫁女 29岁，不会打扮，穿着老土，在公司工作勤勤恳恳。希望自己早日嫁出去，遇到男生不会表达，往往过于急切把对方吓跑了。

你是小萌新，20岁，刚刚进入社会，你的偶像是万人迷，但是并不了解她。但是你无法像万人迷那样游刃有余，而且你还有学生时代的纯真和幻想。喜欢青春偶像，喜欢偶像剧和漂亮的男生，活在对浪漫生活的想象中。

可以测试一下



如果有一段代码在运行，那么可以使用这个，
 在执行的时候会有"wait for it"的提示.

with st.spinner('Wait for it...'):
    time.sleep(5)
st.success('Done!')

In [ ]:
#运行机器人
#如果看不到，直接在终端中运行
! python -m streamlit run chat.streamlit.py

(10) st.file_uploader 文件上传

st.file_uploader(label, type=None, accept_multiple_files=False)
type为文件类型，None为不限。accept_multiple_files是否可以上传多个文件。




In [ ]:
#import os
from io import StringIO #从内存读取字符串
import streamlit as st

# 上传单个文件 Upload single file
uploaded_file = st.file_uploader("Choose a file")
if uploaded_file is not None:
    # To read file as bytes:
    bytes_data = uploaded_file.getvalue()
    st.write(bytes_data)

    # To convert to a string based IO:
    stringio = StringIO(uploaded_file.getvalue().decode("utf-8"))
    st.write(stringio)

    # To read file as string:
    string_data = stringio.read()
    st.write(string_data)

    # Can be used wherever a "file-like" object is accepted:
    dataframe = pd.read_csv(uploaded_file)
    st.write(dataframe)

#上传多个文件 Upload multi files
uploaded_files = st.file_uploader("Choose a CSV file", accept_multiple_files=True)
for uploaded_file in uploaded_files:
    bytes_data = uploaded_file.read()
    st.write("filename:", uploaded_file.name)
    st.write(bytes_data)